In [10]:
import numpy as np
import pandas as pd
import re
import ssl
import nltk
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dinko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dinko\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
netflix_dataset = pd.read_csv('./netflix_titles.csv')
netflix_dataset.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [12]:
netflix_dataset.groupby('type').count()

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
type,,,,,,,,,,,
Movie,6131,6131,5943,5656,5691,6131,6131,6129,6128,6131,6131
TV Show,2676,2676,230,2326,2285,2666,2676,2674,2676,2676,2676


In [13]:
netflix_dataset.describe(include=["object", "bool"])

,show_id,type,title,director,cast,country,date_added,rating,duration,listed_in,description
count,8807,8807,8807,6173,7982,7976,8797,8803,8804,8807,8807
unique,8807,2,8807,4528,7692,748,1767,17,220,514,8775
top,s1,Movie,Dick Johnson Is Dead,Rajiv Chilaka,David Attenborough,United States,"January 1, 2020",TV-MA,1 Season,"Dramas, International Movies","Paranormal activity at a lush, abandoned prope..."
freq,1,6131,1,19,19,2818,109,3207,1793,362,4


In [14]:
netflix_dataset.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [15]:
netflix_dataset = netflix_dataset.dropna(subset=['cast', 'country', 'rating'])
netflix_dataset['director'].fillna('Unkown', inplace = True)
netflix_dataset.isna().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      9
release_year    0
rating          0
duration        3
listed_in       0
description     0
dtype: int64

In [16]:
movies = netflix_dataset[netflix_dataset['type'] == 'Movie'].reset_index()
movies = movies.drop(['index', 'release_year', 'show_id', 'duration', 'date_added', 'description', 'type'], axis=1)
movies.head()

,title,director,cast,country,rating,listed_in
0,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",TV-MA,"Dramas, Independent Movies, International Movies"
1,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,PG-13,"Comedies, Dramas"
2,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",TV-MA,"Dramas, International Movies"
3,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,TV-14,"Comedies, International Movies, Romantic Movies"
4,Grown Ups,Dennis Dugan,"Adam Sandler, Kevin James, Chris Rock, David S...",United States,PG-13,Comedies


In [17]:
tv_shows = netflix_dataset[netflix_dataset['type'] == 'TV Show'].reset_index()
tv_shows = tv_shows.drop(['index', 'release_year', 'show_id', 'duration', 'date_added', 'description', 'type'], axis=1)
tv_shows.head()

,title,director,cast,country,rating,listed_in
0,Blood & Water,Unkown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,TV-MA,"International TV Shows, TV Dramas, TV Mysteries"
1,Kota Factory,Unkown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,TV-MA,"International TV Shows, Romantic TV Shows, TV ..."
2,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,TV-14,"British TV Shows, Reality TV"
3,Dear White People,Unkown,"Logan Browning, Brandon P. Bell, DeRon Horton,...",United States,TV-MA,"TV Comedies, TV Dramas"
4,Falsa identidad,Unkown,"Luis Ernesto Franco, Camila Sodi, Sergio Goyri...",Mexico,TV-MA,"Crime TV Shows, Spanish-Language TV Shows, TV ..."


In [ ]:
movie_genres = []

for i in movies['listed_in']:
    movie_genre = re.split(r', \s*', i)
    movie_genres.append(movie_genre)
    
temp_movie_genre_list = []
for sublist in movie_genres:
    for element in sublist:
        temp_movie_genre_list.append(element)
        
movie_genres_list = sorted(set(temp_movie_genre_list))

binary_movie_genres = [[0] * 0 for i in range(len(set(temp_movie_genre_list)))]

for i in movies['listed_in']:
    k = 0
    for j in movie_genres_list:
        if j.lower() in i.lower():
            binary_movie_genres[k].append(1.0)
        else:
            binary_movie_genres[k].append(0.0)
        k+=1
        
binary_movie_genres = pd.DataFrame(binary_movie_genres).transpose()
        
movie_countries = []

for i in movies['country']:
    movie_country = re.split(r', \s*', i)
    movie_countries.append(movie_country)
    
temp_movie_countries_list = []
for sublist in movie_countries:
    for element in sublist:
        temp_movie_countries_list.append(element)
        
movie_countries_list = sorted(set(temp_movie_countries_list))

binary_movie_countries = [[0] * 0 for i in range(len(set(temp_movie_countries_list)))]

for i in movies['country']:
    k = 0
    for j in movie_countries_list:
        if j.lower() in i.lower():
            binary_movie_countries[k].append(1.0)
        else:
            binary_movie_countries[k].append(0.0)
        k+=1
        
binary_movie_countries = pd.DataFrame(binary_movie_countries).transpose()

movie_ratings = []

for i in movies['rating']:
    movie_ratings.append(i)

movie_ratings_list = sorted(set(movie_ratings))

binary_movie_ratings = [[0] * 0 for i in range(len(set(movie_ratings_list)))]

for i in movies['rating']:
    k = 0
    for j in movie_ratings_list:
        if j.lower() in i.lower():
            binary_movie_ratings[k].append(1.0)
        else:
            binary_movie_ratings[k].append(0.0)
        k+=1
        
binary_movie_ratings = pd.DataFrame(binary_movie_ratings).transpose()

movie_directors = []

for i in movies['director']:
    if pd.notna(i):
        movie_director = re.split(r', \s*', i)
        movie_directors.append(movie_director)
    
temp_movie_directors_list = []
for sublist in movie_directors:
    for element in sublist:
        temp_movie_directors_list.append(element)
        
movie_directors_list = sorted(set(temp_movie_directors_list))

binary_movie_directors = [[0] * 0 for i in range(len(set(temp_movie_directors_list)))]

for i in movies['director']:
    k = 0
    for j in movie_directors_list:
        if i.lower() == 'unknown':
            binary_movie_directors[k].append(0.0)
        elif j.lower() in i.lower():
            binary_movie_directors[k].append(1.0)
        else:
            binary_movie_directors[k].append(0.0)
        k+=1
        
binary_movie_directors = pd.DataFrame(binary_movie_directors).transpose()

movie_actors = []

for i in movies['cast']:
    movie_actor = re.split(r', \s*', i)
    movie_actors.append(movie_actor)
    
temp_movie_actors_list = []
for sublist in movie_actors:
    for element in sublist:
        temp_movie_actors_list.append(element)
        
movie_actors_list = sorted(set(temp_movie_actors_list))

binary_movie_actors = [[0] * 0 for i in range(len(set(temp_movie_actors_list)))]

for i in movies['cast']:
    k = 0
    for j in movie_actors_list:
        if j.lower() in i.lower():
            binary_movie_actors[k].append(1.0)
        else:
            binary_movie_actors[k].append(0.0)
        k+=1
        
binary_movie_actors = pd.DataFrame(binary_movie_actors).transpose()
binary_movies = pd.concat([binary_movie_countries, binary_movie_actors,binary_movie_genres, binary_movie_directors], axis=1, ignore_index=True)
binary_movies

In [ ]:
tv_show_actors = []

for i in tv_shows['cast']:
    tv_show_actor = re.split(r', \s*', i)
    tv_show_actors.append(tv_show_actor)
    
    temp_tv_show_actors_list = []
    for sublist in tv_show_actors:
        for element in sublist: 
            temp_tv_show_actors_list.append(element)
        
tv_show_actors_list = sorted(set(temp_tv_show_actors_list))

binary_tv_show_actors = [[0] * 0 for i in range(len(set(temp_tv_show_actors_list)))]

for i in tv_shows['cast']:
    k = 0
    for j in tv_show_actors_list:
        if j.lower() in i.lower():
            binary_tv_show_actors[k].append(1.0)
        else:
            binary_tv_show_actors[k].append(0.0)
        k+=1

binary_tv_show_actors = pd.DataFrame(binary_tv_show_actors).transpose()

tv_show_countries = []
for i in tv_shows['country']:
    tv_show_country = re.split(r', \s*', i)
    tv_show_countries.append(tv_show_country)
    
temp_tv_show_countries_list = []
for sublist in tv_show_countries:
    for element in sublist: 
        temp_tv_show_countries_list.append(element)
        
tv_show_countries_list = sorted(set(temp_tv_show_countries_list))

binary_tv_show_countries = [[0] * 0 for i in range(len(set(temp_tv_show_countries_list)))]

for i in tv_shows['country']:
    k = 0
    for j in tv_show_countries_list:
        if j.lower() in i.lower():
            binary_tv_show_countries[k].append(1.0)
        else:
            binary_tv_show_countries[k].append(0.0)
        k+=1
        
binary_tv_show_countries = pd.DataFrame(binary_tv_show_countries).transpose()

tv_show_genres = []

for i in tv_shows['listed_in']:
    tv_show_genre = re.split(r', \s*', i)
    tv_show_genres.append(tv_show_genre)
    
temp_tv_show_genres_list = []
for sublist in tv_show_genres:
    for element in sublist:
        temp_tv_show_genres_list.append(element)
        
tv_show_genres_list = sorted(set(temp_tv_show_genres_list))

binary_tv_show_genres = [[0] * 0 for i in range(len(set(temp_tv_show_genres_list)))]

for i in tv_shows['listed_in']:
    k = 0
    for j in tv_show_genres_list:
        if j.lower() in i.lower(): 
            binary_tv_show_genres[k].append(1.0)
        else:
            binary_tv_show_genres[k].append(0.0)
        k+=1
        
binary_tv_show_genres = pd.DataFrame(binary_tv_show_genres).transpose()

tv_show_ratings = []

for i in tv_shows['rating']:
    tv_show_ratings.append(i)
    
tv_show_ratings_list = sorted(set(tv_show_ratings))

binary_tv_show_ratings = [[0] * 0 for i in range(len(set(tv_show_ratings_list)))]

for i in tv_shows['rating']:
    k = 0
    for j in tv_show_ratings_list:
        if j.lower() in i.lower():
            binary_tv_show_ratings[k].append(1.0)
        else:
            binary_tv_show_ratings[k].append(0.0)
        k+=1
        
binary_tv_show_ratings = pd.DataFrame(binary_tv_show_ratings).transpose()
binary_tv_shows = pd.concat([binary_tv_show_countries, binary_tv_show_actors, binary_tv_show_genres], axis=1, ignore_index=True)
binary_tv_shows

In [ ]:
def get_recommendations_using_multiple_feature_similarity(title_input):
    if title_input.lower() in movies['title'].str.lower().values:
        return(get_movie_recommendations_using_multiple_feature_similarity(title_input))
    elif title_input.lower() in tv_shows['title'].str.lower().values:
        return(get_tv_show_recommendations_using_multiple_feature_similarity(title_input))
    else:
        return('Title provided could not be found. Please try with another one')

In [ ]:
def get_movie_recommendations_using_multiple_feature_similarity(title_input):
    movie_cosine_similarity_list = []
    binary_movie_list_one = []
    movie_index = movies[movies['title'].str.lower() == title_input.lower()].index.item()
    for i in binary_movies.iloc[movie_index]:
        binary_movie_list_one.append(i)
    point_one = np.array(binary_movie_list_one).reshape(1, -1)
    point_one = [val for sublist in point_one for val in sublist]
    for j in range(len(movies)):
        binary_movie_list_two = []
        for k in binary_movies.iloc[j]:
            binary_movie_list_two.append(k)
        point_two = np.array(binary_movie_list_two).reshape(1, -1)
        point_two = [val for sublist in point_two for val in sublist]
        dot_product = np.dot(point_one, point_two)
        norm_one = np.linalg.norm(point_one)
        norm_two = np.linalg.norm(point_two)
        cosine_similarity = dot_product / (norm_one * norm_two)
        movie_cosine_similarity_list.append(cosine_similarity)
    movies_temp_copy = movies.copy()
    movies_temp_copy['cosine_similarity'] = movie_cosine_similarity_list
    results = movies_temp_copy.sort_values('cosine_similarity', ascending=False)
    results = results[results['title'].str.lower() != title_input.lower()]
    top_ten_movie_results = results.head(10)
    return(top_ten_movie_results)

In [ ]:
def get_tv_show_recommendations_using_multiple_feature_similarity(title_input):
    tv_show_cosine_similarity_list = []
    binary_tv_show_list = []
    tv_show_index = tv_shows[tv_shows['title'].str.lower() == title_input.lower()].index.item()
    for i in binary_tv_shows.iloc[tv_show_index]:
        binary_tv_show_list.append(i)
    point_one = np.array(binary_tv_show_list).reshape(1, -1)
    point_one = [val for sublist in point_one for val in sublist]
    for j in range(len(tv_shows)):
        binary_tv_show_list = []
        for k in binary_tv_shows.iloc[j]:
            binary_tv_show_list.append(k)
        point_two = np.array(binary_tv_show_list).reshape(1, -1)
        point_two = [val for sublist in point_two for val in sublist]
        dot_product = np.dot(point_one, point_two)
        norm_one = np.linalg.norm(point_one)
        norm_two = np.linalg.norm(point_two)
        cosine_similarity = dot_product / (norm_one * norm_two)
        tv_show_cosine_similarity_list.append(cosine_similarity)
    tv_shows_temp_copy = tv_shows.copy()
    tv_shows_temp_copy['cosine_similarity'] = tv_show_cosine_similarity_list
    results = tv_shows_temp_copy.sort_values('cosine_similarity', ascending=False)
    results = results[results['title'].str.lower() != title_input.lower()]
    top_ten_tv_show_results = results.head(10)
    return(top_ten_tv_show_results)

In [ ]:
get_recommendations_using_multiple_feature_similarity('My Little Pony Equestria Girls: Forgotten Friendship')

In [ ]:
get_recommendations_using_multiple_feature_similarity('13 Reasons Why')

In [ ]:
get_recommendations_using_multiple_feature_similarity('A Go! Go! Cory Carson Christmas')

In [ ]:
movies_with_description = netflix_dataset[netflix_dataset['type'] == 'Movie'].reset_index()
movies_with_description = movies_with_description[['title', 'description']]
movies_with_description.head()

In [ ]:
tv_shows_with_description = netflix_dataset[netflix_dataset['type'] == 'TV Show'].reset_index()
tv_shows_with_description = tv_shows_with_description[['title', 'description']]
tv_shows_with_description.head()

In [ ]:
filtered_movie_descriptions = []

words_from_movies = []

for description_text in movies_with_description['description']:
    text_tokens = word_tokenize(description_text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    words_from_movies.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_movie_descriptions.append(filtered)

words_from_movies = [val for sublist in words_from_movies for val in sublist]
words_from_movies = sorted(set(words_from_movies))
movies_with_description['description_filtered'] = filtered_movie_descriptions
movies_with_description.head()

In [ ]:
filtered_tv_show_descriptions = []
words_from_tv_shows = []

for description_text in tv_shows_with_description['description']:
    text_tokens = word_tokenize(description_text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    words_from_tv_shows.append(tokens_without_sw)
    filtered = (' ').join(tokens_without_sw)
    filtered_tv_show_descriptions.append(filtered)
    
words_from_tv_shows = [val for sublist in words_from_tv_shows for val in sublist]
words_from_tv_shows = sorted(set(words_from_tv_shows))
tv_shows_with_description['description_filtered'] = filtered_tv_show_descriptions
tv_shows_with_description.head()

In [ ]:
binary_movies_with_description = [[0] * 0 for i in range(len(set(words_from_movies)))]

for filtered_desc in movies_with_description['description_filtered']:
    k = 0
    for word in words_from_movies:
        if word in filtered_desc:
            binary_movies_with_description[k].append(1.0)
        else:
            binary_movies_with_description[k].append(0.0)
        k+=1
        
binary_movies_with_description = pd.DataFrame(binary_movies_with_description).transpose()

In [ ]:
binary_tv_shows_with_description = [[0] * 0 for i in range(len(set(words_from_tv_shows)))]

for filtered_desc in tv_shows_with_description['description_filtered']:
    k = 0
    for word in words_from_tv_shows:
        if word in filtered_desc:
            binary_tv_shows_with_description[k].append(1.0)
        else:
            binary_tv_shows_with_description[k].append(0.0)
        k+=1
    
binary_tv_shows_with_description = pd.DataFrame(binary_tv_shows_with_description).transpose()


In [ ]:
def get_recommendations_using_description_similarity(title_input):
    if title_input.lower() in movies_with_description['title'].str.lower().values:
        return(get_movie_recommendations_using_description_similarity(title_input))
    elif title_input.lower() in tv_shows_with_description['title'].str.lower().values:
        return(get_tv_show_recommendations_using_description_similarity(title_input))
    else:
        return('Title provided could not be found. Please try with another one')

In [ ]:
def get_movie_recommendations_using_description_similarity(title_input):
    movie_with_description_cosine_similarity_list = []
    binary_movie_with_description_list_one = []
    movie_index = movies_with_description[movies_with_description['title'].str.lower() == title_input.lower()].index.item()
    for i in binary_movies_with_description.iloc[movie_index]:
        binary_movie_with_description_list_one.append(i)
    point_one = np.array(binary_movie_with_description_list_one).reshape(1, -1)
    point_one = [val for sublist in point_one for val in sublist]    
    for j in range(len(movies_with_description)):
        binary_movie_with_description_list_two = []
        for k in binary_movies_with_description.iloc[j]:
            binary_movie_with_description_list_two.append(k)
        point_two = np.array(binary_movie_with_description_list_two).reshape(1, -1)
        point_two = [val for sublist in point_two for val in sublist]
        dot_product = np.dot(point_one, point_two)
        norm_one = np.linalg.norm(point_one)
        norm_two = np.linalg.norm(point_two)
        cosine_similarity = dot_product / (norm_one * norm_two)
        movie_with_description_cosine_similarity_list.append(cosine_similarity)
    movies_with_description_temp_copy = movies_with_description.copy()
    movies_with_description_temp_copy['cosine_similarity'] = movie_with_description_cosine_similarity_list
    results = movies_with_description_temp_copy.sort_values('cosine_similarity', ascending=False)
    results = results[results['title'].str.lower() != title_input.lower()]
    top_ten_movie_with_description_results = results.head(10)
    return(top_ten_movie_with_description_results)

In [ ]:
def get_tv_show_recommendations_using_description_similarity(title_input):
    tv_show_with_description_cosine_similarity_list = []
    binary_tv_show_with_description_list_one = []
    tv_show_index = tv_shows_with_description[tv_shows_with_description['title'].str.lower() == title_input.lower()].index.item()
    for i in binary_tv_shows_with_description.iloc[tv_show_index]:
        binary_tv_show_with_description_list_one.append(i)
    point_one = np.array(binary_tv_show_with_description_list_one).reshape(1, -1)
    point_one = [val for sublist in point_one for val in sublist]
    for j in range(len(tv_shows_with_description)):
        binary_tv_show_with_description_list_two = []
        for k in binary_tv_shows_with_description.iloc[j]:
            binary_tv_show_with_description_list_two.append(k)
        point_two = np.array(binary_tv_show_with_description_list_two).reshape(1, -1)
        point_two = [val for sublist in point_two for val in sublist]
        dot_product = np.dot(point_one, point_two)
        norm_one = np.linalg.norm(point_one)
        norm_two = np.linalg.norm(point_two)
        cosine_similarity = dot_product / (norm_one * norm_two)
        tv_show_with_description_cosine_similarity_list.append(cosine_similarity)
    tv_shows_with_description_temp_copy = tv_shows_with_description.copy()
    tv_shows_with_description_temp_copy['cosine_similarity'] = tv_show_with_description_cosine_similarity_list
    results = tv_shows_with_description_temp_copy.sort_values('cosine_similarity', ascending=False)
    results = results[results['title'].str.lower() != title_input.lower()]
    top_ten_tv_show_with_description_results = results.head(10)
    return(top_ten_tv_show_with_description_results)

In [ ]:
get_recommendations_using_description_similarity('My Little Pony Equestria Girls: Forgotten Friendship')

In [ ]:
get_recommendations_using_description_similarity('13 Reasons Why')

In [ ]:
get_recommendations_using_description_similarity('A Go! Go! Cory Carson Christmas')